In [1]:
# 全局设置
import os
import datetime as dt

import numpy as np
import pandas as pd

import QuantStudio.api as QS
fd = QS.FactorDB.FactorTools
Factorize = QS.FactorDB.Factorize

# 外部因子库

## TushareDB

API 地址: https://tushare.pro/

继承自 FactorDB, 初始化方法 `__init__`:
* sys_args: dict, 因子库参数集
* config_file: None 或者 str, 因子库的配置文件地址, None 表示使用默认配置文件, 默认文件名为 "TushareDBConfig.json", 默认路径为参见: [配置文件](../基本框架.ipynb#QuantStudio-对象)

参数集:
* 名称: str, 默认值 "TushareDB"
* Token: str, 无默认值必须指定, API Token

当前 TushareDB 的库信息文件路径不能修改, 在 QuantStudio 包目录下 Resource 目录下的 TushareDBInfo.xlsx, 记录了相关配置信息

In [2]:
# TushareDB
TushareDB = QS.FactorDB.TushareDB(sys_args={"Token": "548684efdc7b10eb3514d48dce6b04574d9b689e0765b11438a8f425"})
TushareDB.connect();

print("参数集 : ", TushareDB.Args)

参数集 :  {'名称': 'TushareDB', 'Token': '548684efdc7b10eb3514d48dce6b04574d9b689e0765b11438a8f425'}


### 获取时点序列

#### 获取交易日序列

获取交易日序列 getTradeDay, 输入参数:
* start_date: date 或者 datetime, 默认值 None, 待获取的交易日序列的起始日, None 表示最早的第一个交易日
* end_date: date 或者 datetime, 默认值 None, 待获取的交易日序列的结束日, None 表示今天
* exchange: str, 可选 "SSE"(默认值), "SZSE", "SHFE", "DCE", "CZCE", "INE", "CFFEX", 指定的交易所
* output_type: str, 可选 "date"(默认值), "datetime", 返回的交易日数据类型

In [3]:
# 获取交易日序列
print(TushareDB.getTradeDay(start_date=dt.datetime(2022,1,1), end_date=dt.datetime(2022,1,20), exchange="SSE", output_type="datetime"))

[datetime.datetime(2022, 1, 4, 0, 0), datetime.datetime(2022, 1, 5, 0, 0), datetime.datetime(2022, 1, 6, 0, 0), datetime.datetime(2022, 1, 7, 0, 0), datetime.datetime(2022, 1, 10, 0, 0), datetime.datetime(2022, 1, 11, 0, 0), datetime.datetime(2022, 1, 12, 0, 0), datetime.datetime(2022, 1, 13, 0, 0), datetime.datetime(2022, 1, 14, 0, 0), datetime.datetime(2022, 1, 17, 0, 0), datetime.datetime(2022, 1, 18, 0, 0), datetime.datetime(2022, 1, 19, 0, 0), datetime.datetime(2022, 1, 20, 0, 0)]


### 获取ID序列

#### 股票 ID

待升级

获取股票 ID 序列 getStockID, 输入参数:
* index_id: str, 默认值 "全体A股". 指定的指数代码
* date: None(默认值) 或者 datetime, 指定日, None 表示今天;
* is_current: bool, 默认值 True. False 表示获取上市日期在指定日之前的股票, True 表示上市日期在指定日之前且当前尚未退市的股票;

In [4]:
# 获取股票 ID 序列
print(TushareDB.getStockID(index_id="全体A股", date=None, is_current=True)[:10])

['000001.SZ', '000002.SZ', '000004.SZ', '000005.SZ', '000006.SZ', '000007.SZ', '000008.SZ', '000009.SZ', '000010.SZ', '000011.SZ']


#### 期货 ID

待升级

获取期货 ID 序列 getFutureID, 输入参数:
* future_code: None(默认值), str 或者 tuple(str), 期货代码或者期货代码列表, None 表示所有期货代码
* date: None(默认值) 或者 datetime, 指定日, None 表示今天
* is_current: bool, 默认值 True. False 表示上市日在指定日之前的期货, True 表示上市日在指定日之前且尚未退市的期货
* contract_type: str, 可选 "月合约"(默认值), "连续合约", "所有", 合约类型

In [ ]:
# 获取期货 ID 序列
print(TushareDB.getFutureID(future_code="IF", date=dt.datetime(2022,1,1), is_current=True, contract_type="月合约")[:10])

### 因子表

In [6]:
# TushareDB 因子表
print("因子表 : ", TushareDB.TableNames)
print("===============================================")

Args = {
    "回溯天数": 0,
}

DTs = [dt.datetime(2020,1,1)+dt.timedelta(i) for i in range(9)]
IDs = ["000001.SZ"]

FT = TushareDB.getTable("A股日线行情", args=Args)
print(f"因子表 '{FT.Name}' 的因子 : ", FT.FactorNames)
print("===============================================")
print(FT.readData(factor_names=["最高价", "收盘价"], ids=IDs, dts=DTs).iloc[:, :, 0])

因子表 :  ['交易日历', 'A股基本资料', '上市公司基本信息', 'A股日线行情', 'A股复权因子', 'A股周线行情', 'A股月线行情', 'A股每日指标', 'A股指数成分和权重', '指数基本信息', '指数日线行情', '大盘指数每日指标', '期货合约信息表', '期货日线行情', '每日结算参数', '仓单日报', '公募基金列表', '场内基金日线行情', '公募基金公司', '期权合约信息', '期权日线行情', 'Shibor利率', 'Shibor报价数据', 'LPR贷款基础利率', 'Libor拆借利率', 'Hibor利率']
因子表 'A股日线行情' 的因子 :  ['开盘价', '最高价', '最低价', '收盘价', '昨收价', '涨跌额', '涨跌幅', '成交量(手)', '成交额(千元)', '交易日期']
              最高价    收盘价
2020-01-01    NaN    NaN
2020-01-02  16.95  16.87
2020-01-03  17.31  17.18
2020-01-04    NaN    NaN
2020-01-05    NaN    NaN
2020-01-06  17.34  17.07
2020-01-07  17.28  17.15
2020-01-08  17.05  16.66
2020-01-09  16.93  16.79


## TinySoftDB

继承自 FactorDB, 初始化方法 `__init__`:
* sys_args: dict, 因子库参数集
* config_file: None 或者 str, 因子库的配置文件地址, None 表示使用默认配置文件, 默认文件名为 "TinySoftDBConfig.json", 默认路径为参见: [配置文件](../基本框架.ipynb#QuantStudio-对象)

参数集:
* 名称: str, 默认值 "TinySoftDB"
* 安装目录: str, 无默认值必须指定, TinySoft 的安装目录
* IP地址: str, 默认值 "tsl.tinysoft.com.cn", TinySoft API 的访问地址
* 端口: int, 默认值 443, TinySoft API 的访问端口
* 用户名: str, 默认值 ""
* 密码: str, 默认值 ""
* 库信息文件: 文件路径, 默认值 QuantStudio 包目录下 Resource 目录下的 TinySoftDBInfo.xlsx, 记录了 TinySoft 相关配置信息
* 因子表参数: dict, 默认值 {}, getTable 时传递给因子表的参数集

In [2]:
# TinySoftDB
import QSExt.FactorDataBase.TinySoftDB
TinySoftDB = QSExt.FactorDataBase.TinySoftDB.TinySoftDB()
TinySoftDB.connect();

### 获取时点序列

#### 获取交易日序列

获取交易日序列 getTradeDay, 输入参数:
* start_date: date 或者 datetime, 默认值 None, 待获取的交易日序列的起始日, None 表示最早的第一个交易日
* end_date: date 或者 datetime, 默认值 None, 待获取的交易日序列的结束日, None 表示今天
* exchange: str, 可选 "SSE"(默认值), "SZSE", 指定的交易所
* output_type: str, 可选 "date"(默认值), "datetime", 返回的交易日数据类型

In [3]:
# 获取交易日序列
print(TinySoftDB.getTradeDay(start_date=dt.datetime(2022,1,1), end_date=dt.datetime(2022,1,20), exchange="SSE", output_type="datetime"))

[datetime.datetime(2022, 1, 4, 0, 0), datetime.datetime(2022, 1, 5, 0, 0), datetime.datetime(2022, 1, 6, 0, 0), datetime.datetime(2022, 1, 7, 0, 0), datetime.datetime(2022, 1, 10, 0, 0), datetime.datetime(2022, 1, 11, 0, 0), datetime.datetime(2022, 1, 12, 0, 0), datetime.datetime(2022, 1, 13, 0, 0), datetime.datetime(2022, 1, 14, 0, 0), datetime.datetime(2022, 1, 17, 0, 0), datetime.datetime(2022, 1, 18, 0, 0), datetime.datetime(2022, 1, 19, 0, 0), datetime.datetime(2022, 1, 20, 0, 0)]


### 获取ID序列

#### 股票 ID

待升级

获取股票 ID 序列 getStockID, 输入参数:
* index_id: str, 默认值 "全体A股". 指定的指数代码
* date: None(默认值) 或者 datetime, 指定日, None 表示今天;
* is_current: bool, 默认值 True. False 表示获取上市日期在指定日之前的股票, True 表示上市日期在指定日之前且当前尚未退市的股票;

In [4]:
# 获取股票 ID 序列
print(TinySoftDB.getStockID(index_id="全体A股", date=None, is_current=True)[:10])

['600000.SH', '600004.SH', '600006.SH', '600007.SH', '600008.SH', '600009.SH', '600010.SH', '600011.SH', '600012.SH', '600015.SH']


#### 期货 ID

待升级

获取期货 ID 序列 getFutureID, 输入参数:
* future_code: None(默认值), str 或者 tuple(str), 期货代码或者期货代码列表, None 表示所有期货代码
* date: None(默认值) 或者 datetime, 指定日, None 表示今天
* is_current: bool, 默认值 True. False 表示上市日在指定日之前的期货, True 表示上市日在指定日之前且尚未退市的期货

In [5]:
# 获取期货 ID 序列
print(TinySoftDB.getFutureID(future_code="IF", date=dt.datetime(2022,1,1), is_current=True)[:10])

['IF2201', 'IF2202', 'IF2203', 'IF2206']


### 因子表

In [3]:
# TinySoftDB 因子表
print("因子表 : ", TinySoftDB.TableNames)
print("===============================================")

Args = {
    "周期": 1,
    "周期单位": "d"
}

DTs = [dt.datetime(2020,1,1)+dt.timedelta(i) for i in range(9)]
IDs = ["000001.SZ"]

FT = TinySoftDB.getTable("分时和日线数据", args=Args)
print(f"因子表 '{FT.Name}' 的因子 : ", FT.FactorNames)
print("===============================================")
print(FT.readData(factor_names=["最高价", "最新价"], ids=IDs, dts=DTs).iloc[:, :, 0])

因子表 :  ['交易日历', '分时和日线数据', '交易明细', '股票基本信息', '发行上市', '名称变更', '暂停、终止、恢复上市', '股本结构', '终止上市股份处理方案', '分红送股', '配股', '增发', '十大股东', '十大流通股东', '机构持股', '股东户数', '控股股东及实际控制人', '董事、监事、高管持股变动', '业绩预测', '业绩快报', '股票公告', '股票大宗交易', '融资融券明细', '特别提示-停牌', '交易公开信息', '沪深港通每日成交汇总', '沪深港通每日十大成交活跃股', '沪深港通持股明细', '合并资产负债表', '合并利润分配表', '合并现金流量表', '主要财务指标', '基金基本信息', '业绩比较基准', '基金管理人', '基金经理', '开放式基金费率', '财务指标', '净值增长率与基准比较', '净值', 'ETF 申购赎回-基本信息', '份额变动', 'ETF 申购赎回-成分股', '持股明细', '资产配置', '资产配置(扩展)', '行业配置', '指数信息', '指数成份', '国内生产总值', '工业增加值', '固定资产投资(不含农户)', '房地产开发投资与销售', '社会消费品零售总额', '居民消费价格', '工业生产者价格', '固定资产投资价格', '农产品生产价格', '民间固定资产投资', '人民币存贷款利率', '回购基本信息', '期货品种代码对照表', '期货主力信息', '期货基本信息', '期货结算参数', '期货涨跌停板幅度', '结算会员成交持仓排名', '期货仓单日报-大商所', '期货仓单日报-上期所', '期货仓单日报-郑商所', '国债期货可交割债券', '债券基本信息', '债券派息和利率', '债券发行', '主体评级', '债券评级', '标准券折算率', '可转债条款', '可转债转股价变动', '可转债转股及余额', '可转债回售结果', '可转债赎回结果']
因子表 '分时和日线数据' 的因子 :  ['名称', '最新价', '开盘价', '最高价', '最低价', '成交量', '成交金额', '成交笔数', '上次价', '结算价', '前结算价', '买一价', '买二价', '买三价', '买四